In [2]:
import ffmpeg
import numpy as np

In [6]:
width = 608  # Number of pixels in width
height = 608  # Number of pixels in height
fps = 30  # Frames per second
duration = 5  # Duration of the video in seconds
output_file = 'random_noise_video.avi'

In [8]:
# Generate random noise frames
num_frames = fps * duration
frames = []
for _ in range(num_frames):
    frame = np.random.randint(0, 256, (height, width), dtype=np.uint8)
    frames.append(frame)

In [9]:
frames[0].shape

(608, 608)

In [21]:
# Write frames to video using ffmpeg
(
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='gray', s='{}x{}'.format(width, height), r=fps)
    .output(output_file, pix_fmt='yuv420p', vcodec='libxvid', crf=18)
    .overwrite_output()
    .run(input=np.concatenate(frames, axis=0).tobytes())
)

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with clang version 15.0.7
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1683204443806/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1683204443806/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-gpl --enable-

Error: ffmpeg error (see stderr output for detail)

## Something is not working.. that's not producing a valid .avi file...
- I think libx264 is not compatible with QuickTime player

In [22]:
import ffmpeg
import numpy as np

def generate_random_noise_video(width, height, fps, duration, output_file):
    num_frames = fps * duration
    frames = []
    for _ in range(num_frames):
        frame = np.random.randint(0, 256, (height, width), dtype=np.uint8)
        frames.append(frame)

    # Convert frames to rawvideo format
    video_data = np.stack(frames)
    video_data = video_data.astype(np.uint8)
    video_data = video_data.tobytes()

    # Write frames to .avi video using ffmpeg-python with libxvid codec
    (
        ffmpeg
        .input('pipe:', format='rawvideo', pix_fmt='gray', s=f'{width}x{height}', r=fps)
        .output(output_file, pix_fmt='yuv420p', vcodec='libxvid')
        .overwrite_output()
        .run(input=video_data)
    )


In [23]:
generate_random_noise_video(width=608, height=608, fps=30, duration=5, output_file='raw.avi')

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with clang version 15.0.7
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1683204443806/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1683204443806/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enable-gpl --enable-

Error: ffmpeg error (see stderr output for detail)

In [16]:
import ffmpeg
import numpy as np
import xarray as xr

def load_video_to_dataarray(video_file):
    # Read video using ffmpeg
    (
        ffmpeg
        .input(video_file)
        .output('pipe:', format='rawvideo', pix_fmt='gray')
        .run(capture_stdout=True, capture_stderr=True)
    )

    # Load video frames dynamically
    width, height = 0, 0
    frames = []
    while True:
        in_bytes = ffmpeg.output('pipe:1').stdout.read(1 * width * height)
        if not in_bytes:
            break
        frame = np.frombuffer(in_bytes, np.uint8)
        frame = frame.reshape((height, width))
        frames.append(frame)
        if width == 0 and height == 0:
            height, width = frame.shape

    # Create xarray DataArray
    num_frames = len(frames)
    video_data = np.stack(frames)
    video_da = xr.DataArray(video_data, dims=('frame', 'y', 'x'), coords={'frame': np.arange(num_frames)})

    return video_da


In [24]:
xr = load_video_to_dataarray('raw.avi')

ValueError: Expected at least 1 input stream(s); got 0

## let's try with opencv instead

In [2]:
import numpy as np
import cv2
size = 608, 608
duration = 1
fps = 30
out = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), False)

for _ in range(fps * duration):
    data = np.random.randint(0, 256, size, dtype='uint8')
    out.write(data)
out.release()

## TODO: turn into function and put into neurodatagen

In [ ]:
def generate_rand_vid()